In [1]:
%pwd

'/Users/a06411/Documents/GitHub/nh_lecture/nhbank_dp_2_pyspark'

## 1. 파이스파크에 jdbc 연결 처리 

In [2]:
import os

## 1-1 jdbc 컨피그 세팅 

In [3]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
spark = (SparkSession.builder.appName('pandas-api')
                             .config("spark.driver.host","127.0.0.1") 
                             .config("spark.driver.bindAddress","127.0.0.1")
                             .config("spark.jars","{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
                             .config("spark.driver.extraClassPath","{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
                             .getOrCreate())

23/06/09 19:32:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark

## 1-2 스파크 데이터베이스 확인 

In [11]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



## 데이터 베이스 생성

In [12]:
spark.sql("CREATE DATABASE IF NOT EXISTS customer_db;")

DataFrame[]

In [13]:
spark.sql("Use customer_db;")

DataFrame[]

In [15]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



# 2. 파이스파크에서 sqlite 연결 

## 2-1 파이스파크 판다스 api 에서  sqlite 연결 

In [6]:
import pyspark.pandas as ps

df = ps.read_sql("Band", con="jdbc:sqlite:{}/beat.db".format(os.getcwd()))
df

,id,fname,lname,birthyear
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940


## 2-2 파이스파크에서 직접 sqlite 연결하기 

In [8]:
py_df = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='Band',
                 url='jdbc:sqlite:{}/beat.db'.format(os.getcwd()))\
        .load()

In [9]:
py_df.show()

+---+-----+---------+---------+
| id|fname|    lname|birthyear|
+---+-----+---------+---------+
|  0| Paul|McCartney|     1942|
|  1| John|   Lennon|     1940|
+---+-----+---------+---------+



## 2-3 판다스에서 sqlite db 연결 처리 

In [17]:

import sqlite3

con = sqlite3.connect("../data/beat.db")

In [18]:
query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

In [19]:
pd.read_sql_query(query,con)

,name
0,Band


In [20]:
pd_df = pd.read_sql("select * from Band", con)

## 2-4 데이터프레임을 변환하기 

In [21]:
ps_df = ps.DataFrame(pd_df)

In [22]:
ps_df

,id,fname,lname,birthyear
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940


In [23]:
sp_df = spark.createDataFrame(pd_df)

In [24]:
sp_df.show()

+---+-----+---------+---------+
| id|fname|    lname|birthyear|
+---+-----+---------+---------+
|  0| Paul|McCartney|     1942|
|  1| John|   Lennon|     1940|
+---+-----+---------+---------+



In [21]:
## sp_df.write.mode("overwrite").saveAsTable('Band')

## 3 파이스파크에서 조회한 것을 테이블로 저장

## 3-1 테이블 저장하기

## 파이스파크

In [25]:
py_df.write.mode("overwrite").saveAsTable('Band')

In [26]:
spark.sql("show tables").show()

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|customer_db|     band|      false|
+-----------+---------+-----------+



In [30]:
spark.sql('select * from band').show()

+---+-----+---------+---------+
| id|fname|    lname|birthyear|
+---+-----+---------+---------+
|  0| Paul|McCartney|     1942|
|  1| John|   Lennon|     1940|
+---+-----+---------+---------+



##  판다스 -> 파이스파크 

In [27]:
sp_df.write.mode("overwrite").saveAsTable('Band1')

In [28]:
spark.sql("show tables").show()

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|customer_db|     band|      false|
|customer_db|    band1|      false|
+-----------+---------+-----------+



In [29]:
spark.sql('select * from band1').show()

+---+-----+---------+---------+
| id|fname|    lname|birthyear|
+---+-----+---------+---------+
|  0| Paul|McCartney|     1942|
|  1| John|   Lennon|     1940|
+---+-----+---------+---------+

